In [482]:
import numpy as np
import pandas as pd
from dotenv import load_dotenv
import os
import alpaca_trade_api as tradeapi
from trade import trade_action
from datetime import datetime, timedelta
from sklearn import svm

In [483]:
def market_data( start, end, tickers ):
    load_dotenv()
    alpaca_api_key = os.getenv('ALPACA_API_KEY')
    alpaca_secret_key = os.getenv('ALPACA_SECRET_KEY')

    # Create the Alpaca API object
    alpaca_api = tradeapi.REST(
       alpaca_api_key,
       alpaca_secret_key,
       api_version = 'v2'
    )
    
    # Set timeframe to "1Day" for Alpaca API
    timeframe = "1Day"
    
    start_date = pd.Timestamp(start,tz='America/New_York')
    end_date = pd.Timestamp(end,tz='America/New_York')
           
    # Get number_of_years' worth of historical data for tickers
    data_df = alpaca_api.get_bars(
        tickers,
        timeframe,
        start = start_date.isoformat(),
        end = end_date.isoformat()
    ).df
    if len(data_df) == 0:
        return []
    return data_df

#df_market = market_data( '2023-08-01', '2023-08-21', tickers=['TSLA'] )

## Plan
* Get all ticker data for a year
* Get one ticker, get signal column
* Slice backwards, 15 for pred, 30 for train
* Scale train
* Rolling train, 30 at a time, predict one day at a time, till the end
* Store the prediction in my instr_df

In [484]:
def derive_instr( instrs ):
    buffer = [0]*4
    instr_df = pd.DataFrame()
    for index, instr in instrs.iterrows():
        #print(index, buffer, instr)
        one_row = pd.DataFrame()
        for idx in range(4):
            if buffer[idx]==1 and instr[idx]==-1:
                #print('In sell:', index, idx, -1)
                one_row = pd.DataFrame([[index, instr.index[idx], -1]])
                buffer[idx]=-1
                break
            elif buffer[idx]!=1 and instr[idx]==1:
                #print('In buy:',index, idx, 1)
                one_row = pd.DataFrame([[index, instr.index[idx], 1]])
                buffer[idx]=1
                break

        if len(one_row) == 0:
            one_row = pd.DataFrame([[index, instr.index[0], 0]])
        instr_df = pd.concat([instr_df, one_row])
    
    return instr_df

# my main

## 1. get market data

In [485]:
today = datetime.today().date()
tickers = ['AAPL', 'AMZN', 'MSFT', 'GOOG']
date_from = today - timedelta(days=365)
data_df = market_data(date_from, today, tickers)
print(data_df)
dfs = {}
for ticker in tickers:
    df = data_df[data_df['symbol']==ticker]
    dfs[ticker] = df
dfs                  

                              open      high       low   close    volume  \
timestamp                                                                  
2022-08-31 04:00:00+00:00  160.305  160.5800  157.1400  157.22  87944856   
2022-09-01 04:00:00+00:00  156.640  158.4200  154.6700  157.96  74224560   
2022-09-02 04:00:00+00:00  159.750  160.3620  154.9650  155.81  77074411   
2022-09-06 04:00:00+00:00  156.470  157.0900  153.6900  154.53  73679935   
2022-09-07 04:00:00+00:00  154.825  156.6700  153.6100  155.96  87554883   
...                            ...       ...       ...     ...       ...   
2023-08-25 04:00:00+00:00  321.470  325.3600  318.8001  322.98  21562978   
2023-08-28 04:00:00+00:00  325.660  326.1500  321.7220  323.70  14811065   
2023-08-29 04:00:00+00:00  321.880  328.9835  321.8800  328.41  19284306   
2023-08-30 04:00:00+00:00  328.670  329.8100  326.4450  328.79  15152925   
2023-08-31 04:00:00+00:00  329.200  330.9099  326.7800  327.76  26414006   

           

{'AAPL':                               open     high      low   close    volume  \
 timestamp                                                                
 2022-08-31 04:00:00+00:00  160.305  160.580  157.140  157.22  87944856   
 2022-09-01 04:00:00+00:00  156.640  158.420  154.670  157.96  74224560   
 2022-09-02 04:00:00+00:00  159.750  160.362  154.965  155.81  77074411   
 2022-09-06 04:00:00+00:00  156.470  157.090  153.690  154.53  73679935   
 2022-09-07 04:00:00+00:00  154.825  156.670  153.610  155.96  87554883   
 ...                            ...      ...      ...     ...       ...   
 2023-08-25 04:00:00+00:00  177.380  179.150  175.820  178.61  50888986   
 2023-08-28 04:00:00+00:00  180.090  180.590  178.545  180.19  43873741   
 2023-08-29 04:00:00+00:00  179.695  184.900  179.500  184.12  53050989   
 2023-08-30 04:00:00+00:00  184.940  187.850  184.740  187.65  60571871   
 2023-08-31 04:00:00+00:00  187.840  189.120  187.480  187.87  60488715   
 
               

## 2. prep data for algo

In [446]:
# signals is a df globally
# construct singals data:
signals_dfs = {}
first = True
for key in dfs:
    signals = dfs[key].loc[:,['vwap']]
    signals['daily_return'] = signals.pct_change()
    signals['lagged_daily_return'] = signals['daily_return'].shift(1)
    signals['twice_lagged'] = signals['daily_return'].shift(2)
    signals.dropna(inplace=True)
    signals_dfs[key]=signals
    if first:
        visual = signals[['daily_return','lagged_daily_return','twice_lagged']].hvplot(
            title=key,
            frame_width=700)
        first = False
    else:
        visual = ( visual +
                  signals[['daily_return','lagged_daily_return','twice_lagged']].hvplot(
                      title=key,
                      frame_width=700)
                 )
        
# visualize data prep for anz later
visual

:Layout
   .NdOverlay.I   :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .NdOverlay.II  :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .NdOverlay.III :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .NdOverlay.IV  :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)

## 3. derive signal based on the observation of data

In [453]:
# derive signal - the singals df is global
for key in signals_dfs:
    # signals_dfs[key]['signal'] = np.where(
    #     (signals_dfs[key]['lagged_daily_return']>0) &
    #     (signals_dfs[key]['twice_lagged']>0), 
    #         -1, 
    #         np.where(
    #             (signals['lagged_daily_return']<0) &
    #             (signals['twice_lagged']<0), 1, 0 
    #                 )
    # )
    signals_dfs[key]['signal'] = np.where(
        signals_dfs[key]['lagged_daily_return']<0, 1, -1)


## 4. SVR on pct_change of vwap

### 4.1 the SVR ML

In [457]:
def one_cycle( cycle ):
    """ one cycle of 45 (30+15) days, using SVR directly on vwap pct_change """
   
    # use lagged returns to train SVM
    data = cycle.drop(['vwap','daily_return','signal'],axis=1)
    label = cycle['signal']
    instrs = pd.DataFrame()
    model = svm.SVR()
    
    # rolling train for 30 days then predict 1 day, 15 cycles for 15 instructions
    for instri in range(0,15):
        iloc_to = instri + 90
        X = data.iloc[:iloc_to,:]
        y = label.iloc[:iloc_to]
        model.fit(X, y)
        signal = model.predict(data.iloc[iloc_to:iloc_to+1,:])
        
        if signal > 0:
            instr = 1
        else:
            instr = -1
        #one_row = pd.DataFrame([[signals.index[iloc_to].date(), cycle['symbol'][0], instr]])
        one_row = pd.DataFrame([[cycle.index[iloc_to].date(), instr]])
        instrs = pd.concat([instrs,one_row], join='outer')
        
    return instrs

### 4.2 One cycle of 45 days (30+15)

In [486]:
instrs = pd.DataFrame()
for key in signals_dfs:
    cycle = signals_dfs[key].iloc[-105:,:]
    cycle_signals = one_cycle(cycle).set_index(0)
    cycle_signals.columns = [key]
    #instrs = pd.concat([instrs, one_cycle(cycle)])
    instrs = pd.concat([instrs, cycle_signals],axis=1)

instrs

,AAPL,AMZN,MSFT,GOOG
0,,,,
2023-08-10,1,1,1,1
2023-08-11,-1,-1,-1,-1
2023-08-14,1,1,1,1
2023-08-15,-1,-1,-1,-1
2023-08-16,1,1,-1,1
2023-08-17,1,1,1,1
2023-08-18,1,1,1,-1
2023-08-21,1,1,1,1
2023-08-22,-1,-1,-1,-1


#### 4.2.1 15-day trade actions

In [459]:
instr_df = derive_instr(instrs)
instr_df

,0,1,2
0,2023-08-10,AAPL,1
0,2023-08-11,AAPL,-1
0,2023-08-14,AAPL,1
0,2023-08-15,AAPL,-1
0,2023-08-16,AAPL,1
0,2023-08-17,AMZN,1
0,2023-08-18,MSFT,1
0,2023-08-21,GOOG,1
0,2023-08-22,AAPL,-1
0,2023-08-23,AMZN,-1


#### 4.2.2 show result

In [487]:
trade_action(instr_df)

On 2023-08-10 trade AAPL
{'action': 'buy', 'price': 178.95, 'bal': 7673.65, 'share': 13, 'status': 0, 'msg': 'success'}
On 2023-08-11 trade AMZN
{'action': 'buy', 'price': 138.03, 'bal': 5189.11, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-14 trade GOOG
{'action': 'buy', 'price': 130.8, 'bal': 2703.91, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-15 trade MSFT
{'action': 'buy', 'price': 322.71, 'bal': 444.94, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-16 trade MSFT
{'action': 'sell', 'price': 321.36, 'bal': 2694.46, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-17 trade AMZN
{'action': 'sell', 'price': 134.76, 'bal': 5120.14, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-18 trade AMZN
{'action': 'buy', 'price': 132.52, 'bal': 2734.78, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-21 trade AAPL
{'action': 'none', 'price': 175.19, 'bal': 2734.78, 'share': 13, 'status': -1, 'msg': 'invalid action (1/-1 only)'}
On 2023-08-22 trade M

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

## 5. SVC on scaled all data

In [ ]:
# shifted is a df globally. shift(3) is required to match signal (daily return, lagged, twice_lagged)
# shifted data is for 
shifted_dfs = {}
for key in ticker_data:
    shifted_dfs[key] = dfs[key].drop('symbol',axis=1).shift(3).dropna()

### 5.1 the SVC ML

In [488]:
from sklearn.preprocessing import StandardScaler

def svc_ML( data, label ):
    """ one cycle of 45 (30+15) days, using SVR on all cols """
   
    # scale for X
    scaler = StandardScaler()
    scaler.fit(data)
    all_data = pd.DataFrame(scaler.transform(data))
    all_label = label
    
    instrs = pd.DataFrame()
    model = svm.SVC(kernel='rbf')
    
    # rolling train for 30 days then predict 1 day, 15 cycles for 15 instructions
    for instri in range(0,15):
        iloc_to = instri + 90
        X = all_data.iloc[:iloc_to,:]
        y = all_label.iloc[:iloc_to]
        model.fit(X, y)
        signal = model.predict(all_data.iloc[iloc_to:iloc_to+1,:])
        instr = signal[0]
        one_row = pd.DataFrame([[data.index[iloc_to].date(), instr]])
        instrs = pd.concat([instrs,one_row], join='outer')
        
    return instrs

### 5.2 one cycle of 45 days

In [489]:
instrs = pd.DataFrame()
for key in shifted_dfs:
    cycle = shifted_dfs[key].iloc[-105:,:]
    data = cycle
    label = signals_dfs[key]['signal']
    cycle_signals = svr_ML(data, label).set_index(0)
    cycle_signals.columns = [key]
    #instrs = pd.concat([instrs, one_cycle(cycle)])
    instrs = pd.concat([instrs, cycle_signals],axis=1)

instrs

,AAPL,AMZN,MSFT,GOOG
0,,,,
2023-08-10,1,-1,1,1
2023-08-11,1,1,-1,1
2023-08-14,1,1,-1,1
2023-08-15,1,1,1,1
2023-08-16,1,1,-1,1
2023-08-17,1,-1,-1,1
2023-08-18,1,1,1,1
2023-08-21,1,1,-1,1
2023-08-22,1,1,1,1


#### 5.2.1 15-day trade actions

In [490]:
instr_df = derive_instr(instrs)
instr_df

,0,1,2
0,2023-08-10,AAPL,1
0,2023-08-11,AMZN,1
0,2023-08-14,GOOG,1
0,2023-08-15,MSFT,1
0,2023-08-16,MSFT,-1
0,2023-08-17,AMZN,-1
0,2023-08-18,AMZN,1
0,2023-08-21,AAPL,0
0,2023-08-22,MSFT,1
0,2023-08-23,AAPL,-1


#### 5.2.2 show result

In [481]:
trade_action(instr_df)

On 2023-08-10 trade AAPL
{'action': 'buy', 'price': 178.95, 'bal': 7673.65, 'share': 13, 'status': 0, 'msg': 'success'}
On 2023-08-11 trade AMZN
{'action': 'buy', 'price': 138.03, 'bal': 5189.11, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-14 trade GOOG
{'action': 'buy', 'price': 130.8, 'bal': 2703.91, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-15 trade MSFT
{'action': 'buy', 'price': 322.71, 'bal': 444.94, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-16 trade MSFT
{'action': 'sell', 'price': 321.36, 'bal': 2694.46, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-17 trade AMZN
{'action': 'sell', 'price': 134.76, 'bal': 5120.14, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-18 trade AMZN
{'action': 'buy', 'price': 132.52, 'bal': 2734.78, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-21 trade AAPL
{'action': 'none', 'price': 175.19, 'bal': 2734.78, 'share': 13, 'status': -1, 'msg': 'invalid action (1/-1 only)'}
On 2023-08-22 trade M

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

## 6. svr on pct_change of all columns

In [ ]:
# shifted is a df globally. first we cal pct_change, then shift(2) is required to match signal (daily return, lagged, twice_lagged)
shifted_dfs = {}
for key in ticker_data:
    shifted_dfs[key] = dfs[key].drop('symbol',axis=1).pct_change().shift(2)
    shifted_dfs[key].dropna(inplace=True)

shifted_dfs

### 6.1 train and predict

In [491]:
def one_cycle( data, label ):
    """ one cycle of 45 (30+15) days, using SVR directly on vwap pct_change """
   
    # use lagged returns to train SVM
    instrs = pd.DataFrame()
    model = svm.SVR()
    
    # rolling train for 30 days then predict 1 day, 15 cycles for 15 instructions
    for instri in range(0,15):
        iloc_to = instri + 30
        X = data.iloc[:iloc_to,:]
        y = label.iloc[:iloc_to]
        model.fit(X, y)
        signal = model.predict(data.iloc[iloc_to:iloc_to+1,:])
        
        if signal > 0:
            instr = 1
        else:
            instr = -1
        #one_row = pd.DataFrame([[signals.index[iloc_to].date(), cycle['symbol'][0], instr]])
        one_row = pd.DataFrame([[cycle.index[iloc_to].date(), instr]])
        instrs = pd.concat([instrs,one_row], join='outer')
        
    return instrs

### 6.2 one cycle of 45 days

In [492]:
instrs = pd.DataFrame()
for key in shifted_dfs:
    cycle = shifted_dfs[key].iloc[-45:,:]
    cycle_signals = one_cycle(cycle, signals_dfs[key]['signal']).set_index(0)
    cycle_signals.columns = [key]
    instrs = pd.concat([instrs, cycle_signals],axis=1)

instrs

,AAPL,AMZN,MSFT,GOOG
0,,,,
2023-08-10,-1,1,1,1
2023-08-11,1,1,1,1
2023-08-14,1,1,1,1
2023-08-15,1,1,-1,1
2023-08-16,1,1,1,1
2023-08-17,1,1,-1,1
2023-08-18,1,1,-1,1
2023-08-21,1,1,-1,1
2023-08-22,-1,1,-1,1


#### 6.2.1 15 day trade action

In [493]:
instr_df = derive_instr(instrs)
instr_df

,0,1,2
0,2023-08-10,AMZN,1
0,2023-08-11,AAPL,1
0,2023-08-14,MSFT,1
0,2023-08-15,MSFT,-1
0,2023-08-16,MSFT,1
0,2023-08-17,MSFT,-1
0,2023-08-18,GOOG,1
0,2023-08-21,AAPL,0
0,2023-08-22,AAPL,-1
0,2023-08-23,AAPL,0


#### 6.2.2 show result

In [494]:
trade_action(instr_df)

On 2023-08-10 trade AMZN
{'action': 'buy', 'price': 138.88, 'bal': 7500.16, 'share': 18, 'status': 0, 'msg': 'success'}
On 2023-08-11 trade AAPL
{'action': 'buy', 'price': 177.57, 'bal': 5014.18, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-14 trade MSFT
{'action': 'buy', 'price': 322.39, 'bal': 2757.45, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-15 trade MSFT
{'action': 'sell', 'price': 322.71, 'bal': 5016.42, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-16 trade MSFT
{'action': 'buy', 'price': 321.36, 'bal': 2766.9, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-17 trade MSFT
{'action': 'sell', 'price': 318.88, 'bal': 4999.06, 'share': 7, 'status': 0, 'msg': 'success'}
On 2023-08-18 trade GOOG
{'action': 'buy', 'price': 128.5, 'bal': 2557.56, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-21 trade AAPL
{'action': 'none', 'price': 175.19, 'bal': 2557.56, 'share': 14, 'status': -1, 'msg': 'invalid action (1/-1 only)'}
On 2023-08-22 trade AAP

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

## 7. random trade action

### 7.1 randomly generate trade actions

In [420]:
instrs = np.random.randn(50, 1)

def sample_df():
    instri = 0

    # using alpaca data for the dates to avoid non-trading days
    df_market = dfs[tickers[0]].iloc[-15:,:]
    
    df = pd.DataFrame()
    for index, data in df_market.iterrows():
        if instrs[instri] > 0:
            instr = 1
        else:
            instr = -1
        instri += 1
        one_row = pd.DataFrame([[index.date(), tickers[np.random.randint(4)], instr]])
        df = pd.concat([df,one_row], join='outer')
    return df

random_actions = sample_df()
random_actions

,0,1,2
0,2023-08-10,AAPL,-1
0,2023-08-11,AAPL,1
0,2023-08-14,GOOG,-1
0,2023-08-15,AMZN,-1
0,2023-08-16,AMZN,-1
0,2023-08-17,AAPL,1
0,2023-08-18,GOOG,1
0,2023-08-21,AMZN,-1
0,2023-08-22,MSFT,-1
0,2023-08-23,MSFT,-1


### 7.2 show result

In [421]:
trade_action(random_actions)

On 2023-08-10 trade AAPL
{'action': 'none', 'price': 178.95, 'bal': 10000.0, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-11 trade AAPL
{'action': 'buy', 'price': 177.57, 'bal': 7514.02, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-14 trade GOOG
{'action': 'none', 'price': 130.8, 'bal': 7514.02, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-15 trade AMZN
{'action': 'none', 'price': 139.06, 'bal': 7514.02, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-16 trade AMZN
{'action': 'none', 'price': 136.14, 'bal': 7514.02, 'share': 0, 'status': -1, 'msg': 'no share to sell'}
On 2023-08-17 trade AAPL
{'action': 'buy', 'price': 175.53, 'bal': 5056.6, 'share': 14, 'status': 0, 'msg': 'success'}
On 2023-08-18 trade GOOG
{'action': 'buy', 'price': 128.5, 'bal': 2615.1, 'share': 19, 'status': 0, 'msg': 'success'}
On 2023-08-21 trade AMZN
{'action': 'none', 'price': 134.08, 'bal': 2615.1, 'share': 0, 'status': -1, 'msg': 'no share to se

:Layout
   .NdOverlay.I  :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .NdOverlay.II :NdOverlay   [Variable]
      :Curve   [date]   (value)

### 7.3 run 500 times random actions see distribution

In [422]:
final_worths = []
for n in range(500):
    final_worths.append(trade_action( sample_df(), verbose=0 ))
pd.DataFrame(final_worths).hvplot(kind='hist')

:Histogram   [0]   (0_count)